<a href="https://colab.research.google.com/github/Ivan8Garcia/Proyecto_NoCountry/blob/main/Vuelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

datos=pd.read_csv("/content/flight_delays.csv")
datos.shape

(1747627, 16)

In [19]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 16 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   FlightID            int64 
 1   Airline             object
 2   FlightNumber        int64 
 3   Origin              object
 4   Destination         object
 5   ScheduledDeparture  object
 6   ActualDeparture     object
 7   ScheduledArrival    object
 8   ActualArrival       object
 9   DelayMinutes        int64 
 10  DelayReason         object
 11  Cancelled           bool  
 12  Diverted            bool  
 13  AircraftType        object
 14  TailNumber          object
 15  Distance            int64 
dtypes: bool(2), int64(4), object(10)
memory usage: 190.0+ MB


In [20]:
datos.isnull().sum()

,0
FlightID,0
Airline,0
FlightNumber,0
Origin,0
Destination,0
ScheduledDeparture,0
ActualDeparture,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0


In [21]:
datos.fillna(0,inplace=True)

In [22]:
datos.isnull().sum()

,0
FlightID,0
Airline,0
FlightNumber,0
Origin,0
Destination,0
ScheduledDeparture,0
ActualDeparture,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0


In [23]:
datos.head()

,FlightID,Airline,FlightNumber,Origin,Destination,ScheduledDeparture,ActualDeparture,ScheduledArrival,ActualArrival,DelayMinutes,DelayReason,Cancelled,Diverted,AircraftType,TailNumber,Distance
0,1,United,4558,ORD,MIA,2024-09-01 08:11,2024-09-01 08:30,2024-09-01 12:11,2024-09-01 12:19,8,Weather,True,False,Boeing 737,N71066,1031
1,2,Delta,8021,LAX,MIA,2024-09-01 10:25,2024-09-01 10:41,2024-09-01 13:25,2024-09-01 13:27,2,Air Traffic Control,True,True,Airbus A320,N22657,1006
2,3,Southwest,7520,DFW,SFO,2024-09-01 16:53,2024-09-01 17:05,2024-09-01 17:53,2024-09-01 18:07,14,Weather,True,True,Boeing 737,N95611,2980
3,4,Delta,2046,ORD,BOS,2024-09-01 14:44,2024-09-01 15:04,2024-09-01 18:44,2024-09-01 18:34,-10,0,False,False,Boeing 777,N90029,1408
4,5,Delta,6049,LAX,SEA,2024-09-01 01:51,2024-09-01 02:08,2024-09-01 05:51,2024-09-01 06:15,24,Air Traffic Control,False,True,Boeing 737,N27417,2298


##ELIMINACION DE COLUMNAS PARA MODELO MVP, SOLO NECESITAREMOS "Aerolínea","Origen", "Destino", "Fecha/Hora", "Retraso".

In [24]:
datos = datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture',]]
datos = datos.rename(columns={'Airline': 'Aerolínea', 'Origin': 'Origen', 'Destination': 'Destino', 'ScheduledDeparture': 'Fecha/Hora'})
datos['Fecha/Hora'] = pd.to_datetime(datos['Fecha/Hora'])

# Add the day of the week column in Spanish
# First, get the day name in English
datos['Dia_semana_temp'] = datos['Fecha/Hora'].dt.day_name()

# Map to Spanish
spanish_days = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

datos['Dia_semana'] = datos['Dia_semana_temp'].map(spanish_days)

# Drop the temporary column
datos.drop('Dia_semana_temp', axis=1, inplace=True)

datos['Fecha'] = datos['Fecha/Hora'].dt.date
datos['Hora'] = datos['Fecha/Hora'].dt.time
datos.drop('Fecha/Hora', axis=1, inplace=True)

datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora
0,United,ORD,MIA,Domingo,2024-09-01,08:11:00
1,Delta,LAX,MIA,Domingo,2024-09-01,10:25:00
2,Southwest,DFW,SFO,Domingo,2024-09-01,16:53:00
3,Delta,ORD,BOS,Domingo,2024-09-01,14:44:00
4,Delta,LAX,SEA,Domingo,2024-09-01,01:51:00


In [25]:
import pandas as pd

# Re-reading the data to ensure DelayMinutes is present
temp_datos = pd.read_csv("/content/flight_delays.csv")

# Fill NaN values in DelayReason as done previously, or any other NaNs that might affect numerical columns
temp_datos.fillna(0, inplace=True)

# Select the required columns including DelayMinutes
datos = temp_datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture', 'DelayMinutes']].copy()

# Rename columns to Spanish
datos = datos.rename(columns={
    'Airline': 'Aerolínea',
    'Origin': 'Origen',
    'Destination': 'Destino',
    'ScheduledDeparture': 'Fecha/Hora',
    'DelayMinutes': 'Retraso_minutos'
})

# Convert 'Fecha/Hora' to datetime
datos['Fecha/Hora'] = pd.to_datetime(datos['Fecha/Hora'])

# Add the day of the week column in Spanish
datos['Dia_semana_temp'] = datos['Fecha/Hora'].dt.day_name()
spanish_days = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
datos['Dia_semana'] = datos['Dia_semana_temp'].map(spanish_days)

# Create the binary 'retrasado' column based on 'Retraso_minutos'
datos['Retrasado'] = (datos['Retraso_minutos'] > 15).astype(int)

# Split Fecha/Hora into Fecha and Hora
datos['Fecha'] = datos['Fecha/Hora'].dt.date
datos['Hora'] = datos['Fecha/Hora'].dt.time

# Drop temporary and original Fecha/Hora column
datos.drop(columns=['Fecha/Hora', 'Dia_semana_temp'], inplace=True)

# Ensure the requested column order, placing 'Retraso_minutos' and 'Retrasado' after 'Hora'
datos = datos[['Aerolínea', 'Origen', 'Destino', 'Dia_semana', 'Fecha', 'Hora', 'Retraso_minutos', 'Retrasado']]

datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
0,United,ORD,MIA,Domingo,2024-09-01,08:11:00,8,0
1,Delta,LAX,MIA,Domingo,2024-09-01,10:25:00,2,0
2,Southwest,DFW,SFO,Domingo,2024-09-01,16:53:00,14,0
3,Delta,ORD,BOS,Domingo,2024-09-01,14:44:00,-10,0
4,Delta,LAX,SEA,Domingo,2024-09-01,01:51:00,24,1


Entregables deseados

Notebook (Jupyter/Colab) del equipo de Data Science, que contenga:

Exploración y limpieza de datos (EDA);

Creación de variables relevantes (ej.: hora del vuelo, día de la semana, aeropuerto de origen/destino, aerolínea);

Entrenamiento de un modelo predictivo (ej.: Logistic Regression, Random Forest);

Evaluación del desempeño (Accuracy, Precision, Recall, F1-score);

Exportación del modelo serializado (joblib/pickle).

In [26]:
datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
0,United,ORD,MIA,Domingo,2024-09-01,08:11:00,8,0
1,Delta,LAX,MIA,Domingo,2024-09-01,10:25:00,2,0
2,Southwest,DFW,SFO,Domingo,2024-09-01,16:53:00,14,0
3,Delta,ORD,BOS,Domingo,2024-09-01,14:44:00,-10,0
4,Delta,LAX,SEA,Domingo,2024-09-01,01:51:00,24,1


In [27]:
datos['Retraso_minutos'] = datos['Retraso_minutos'].astype('float64')
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Aerolínea        object 
 1   Origen           object 
 2   Destino          object 
 3   Dia_semana       object 
 4   Fecha            object 
 5   Hora             object 
 6   Retraso_minutos  float64
 7   Retrasado        int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 106.7+ MB


In [28]:
# Convert 'Hora' from datetime.time object to a float representing total hours
datos['Hora'] = datos['Hora'].apply(lambda x: x.hour + x.minute/60 + x.second/3600)

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Aerolínea        object 
 1   Origen           object 
 2   Destino          object 
 3   Dia_semana       object 
 4   Fecha            object 
 5   Hora             float64
 6   Retraso_minutos  float64
 7   Retrasado        int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 106.7+ MB


In [29]:
datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
0,United,ORD,MIA,Domingo,2024-09-01,8.183333,8.0,0
1,Delta,LAX,MIA,Domingo,2024-09-01,10.416667,2.0,0
2,Southwest,DFW,SFO,Domingo,2024-09-01,16.883333,14.0,0
3,Delta,ORD,BOS,Domingo,2024-09-01,14.733333,-10.0,0
4,Delta,LAX,SEA,Domingo,2024-09-01,1.850000,24.0,1


In [30]:
datos.tail()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
1747622,United,DFW,MIA,Domingo,2024-09-01,12.766667,-5.0,0
1747623,United,ATL,SEA,Domingo,2024-09-01,20.716667,20.0,1
1747624,Delta,JFK,SFO,Domingo,2024-09-01,3.433333,-6.0,0
1747625,Delta,JFK,SEA,Domingo,2024-09-01,8.066667,16.0,1
1747626,United,LAX,MIA,Domingo,2024-09-01,12.766667,19.0,1


In [31]:
#Todas estas columnas son categoricas (OBJ), para que se puedan hacer buenos modelos hay que codificarlas
categoricas= ["Aerolínea","Origen","Destino","Dia_semana"]

pd.get_dummies(data=datos,columns=categoricas,dtype=int).head(5)

,Fecha,Hora,Retraso_minutos,Retrasado,Aerolínea_American Airlines,Aerolínea_Delta,Aerolínea_Southwest,Aerolínea_United,Origen_ATL,Origen_DFW,Origen_JFK,Origen_LAX,Origen_ORD,Destino_BOS,Destino_JFK,Destino_MIA,Destino_SEA,Destino_SFO,Dia_semana_Domingo
0,2024-09-01,8.183333,8.0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
1,2024-09-01,10.416667,2.0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1
2,2024-09-01,16.883333,14.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1
3,2024-09-01,14.733333,-10.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
4,2024-09-01,1.850000,24.0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1


In [32]:
datos_codificados=pd.get_dummies(data=datos,columns=categoricas,dtype=int)
datos_codificados.sample(5)

,Fecha,Hora,Retraso_minutos,Retrasado,Aerolínea_American Airlines,Aerolínea_Delta,Aerolínea_Southwest,Aerolínea_United,Origen_ATL,Origen_DFW,Origen_JFK,Origen_LAX,Origen_ORD,Destino_BOS,Destino_JFK,Destino_MIA,Destino_SEA,Destino_SFO,Dia_semana_Domingo
39053,2024-09-01,4.000000,22.0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1
710642,2024-09-01,19.350000,17.0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1
788370,2024-09-01,4.850000,20.0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1
230128,2024-09-01,6.816667,24.0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1
1534507,2024-09-01,22.300000,-3.0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1


In [33]:
#si se utiliza los dos datos de la celda superior podemos sesgar el modelo, asi que solo debemos usar 1 de ellas
datos_codificados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 19 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Fecha                        object 
 1   Hora                         float64
 2   Retraso_minutos              float64
 3   Retrasado                    int64  
 4   Aerolínea_American Airlines  int64  
 5   Aerolínea_Delta              int64  
 6   Aerolínea_Southwest          int64  
 7   Aerolínea_United             int64  
 8   Origen_ATL                   int64  
 9   Origen_DFW                   int64  
 10  Origen_JFK                   int64  
 11  Origen_LAX                   int64  
 12  Origen_ORD                   int64  
 13  Destino_BOS                  int64  
 14  Destino_JFK                  int64  
 15  Destino_MIA                  int64  
 16  Destino_SEA                  int64  
 17  Destino_SFO                  int64  
 18  Dia_semana_Domingo           int64  
dtype